In [ ]:
import importlib
import scripts
importlib.reload(scripts.dataset_generator)
from scripts.dataset_generator import *


In [ ]:
dataset = create_tf_dataset('../data/extracted_stems/MIDI')
print(dataset)

In [ ]:
total_samples = 10000  # Replace with the actual count
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)
test_size = total_samples - train_size - val_size
batch_size = 32

In [ ]:
dataset = dataset.shuffle(buffer_size=10000, reshuffle_each_iteration=False)

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size).take(test_size)

train_dataset = train_dataset.repeat().batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.repeat().batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.repeat().batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
from models.pitch_classifier import build_pitch_classifier_cnn

model = build_pitch_classifier_cnn()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,  # Adjust based on your needs
    verbose=1
)